# Initialization

## modules

In [2]:
from case_study.integrations.database import  connections
from case_study.integrations.duckdb import DuckDBIntegrator
from case_study.ai_join_advisor import SmartJoinBuilder
from case_study.jupyter_ui import select_databases_widget,select_tables_widget

2025-04-24 07:47:54,428 - database-queries - INFO - Initializing ConnectionConfig with path: config.yaml
2025-04-24 07:47:54,576 - database-queries - INFO - Loading configuration from config.yaml
2025-04-24 07:47:54,703 - database-queries - INFO - Processing connection configuration for: postgres_datamancers
2025-04-24 07:47:54,803 - database-queries - INFO - Successfully configured connection: postgres_datamancers
2025-04-24 07:47:54,915 - database-queries - INFO - Processing connection configuration for: datamancers
2025-04-24 07:47:55,015 - database-queries - INFO - Successfully configured connection: datamancers
2025-04-24 07:47:55,099 - database-queries - INFO - ConnectionConfig initialization completed


## Required objects
- SmartJoinBuilder() - class that suggest join between two selected tables
- DuckDbIntegrator() - class for querying across multiple (types) of databases within single query  

In [3]:
builder = SmartJoinBuilder()
query_engine = DuckDBIntegrator()

2025-04-24 07:47:57,130 - join-advisor - INFO - Initializing SmartJoinBuilder with config path: config.yaml
2025-04-24 07:47:57,341 - duckdb-integrator - INFO - Initializing DuckDB integrator with config path: config.yaml
2025-04-24 07:47:57,482 - duckdb-integrator - INFO - Using DuckDB database path: data/duckdb_logs.db
2025-04-24 07:47:57,557 - duckdb-integrator - INFO - Ensured database directory exists: data
2025-04-24 07:47:57,772 - duckdb-integrator - INFO - DuckDB connection established
2025-04-24 07:47:57,873 - database-queries - INFO - Initializing ConnectionConfig with path: config.yaml
2025-04-24 07:47:57,946 - database-queries - INFO - Loading configuration from config.yaml
2025-04-24 07:47:58,064 - database-queries - INFO - Processing connection configuration for: postgres_datamancers
2025-04-24 07:47:58,147 - database-queries - INFO - Successfully configured connection: postgres_datamancers
2025-04-24 07:47:58,237 - database-queries - INFO - Processing connection configur

# Workflow
1. select databases to connect from your configuration file 
2. Smart Join Suggest (optional): do you want to join two tables across your connected databases but don't know how? Select them and let the ai give you the suggestion!
3. Querying engine: Run your query across databases! You can query any data across your connections, just wrap the table reference into "DB type function" (e.g. BIGQUERY(project-id.dataset_id.table_id)) and let the magic flow! You can also join tables across databases!

In [ ]:
select_databases_widget(connections,builder)

In [ ]:
select_tables_widget(builder)

In [8]:
query="""WITH
    bq_data AS (
        SELECT *, EXTRACT(EPOCH FROM (ts_end-ts_start)) / 3600.0 AS duration 
        FROM BIGQUERY('datamancers.dbt.L1_POSTGRES__JOBS__planned_cleaned')
    ), 
    
    pg_data_resources AS (
        SELECT * FROM POSTGRES('public.project_resources_pricings')
    ),
    
    pg_data_pricings AS (
        SELECT * FROM POSTGRES('public.pricing_client')
    ),
    
    pg_data AS (
        SELECT pg_data_resources.*, pg_data_pricings.value AS pricing_value FROM pg_data_resources 
        LEFT JOIN pg_data_pricings
        ON pg_data_resources.pricing_client_id = pg_data_pricings.id
    ),

    final_cross_database_join AS (

        SELECT *, bq_data.duration * pg_data.pricing_value AS total_amount FROM bq_data 
        INNER JOIN pg_data 
        ON pg_data.project_id = bq_data.project_id
        AND pg_data.client_id = bq_data.client_id
        AND pg_data.resource_id = bq_data.resource_id
        AND bq_data.ts_start BETWEEN pg_data.date_from AND COALESCE(pg_data.date_to, CURRENT_DATE)
    )

    SELECT * FROM final_cross_database_join
        """

data=query_engine.run_query(query)
data

/Users/vitmrnavek/projects/datamancers/clients/rohlik/case_study/venv/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


2025-04-24 07:51:16,331 - duckdb-integrator - INFO - Logging query performance for source: bigquery
2025-04-24 07:51:16,417 - duckdb-integrator - INFO - Query performance logged successfully
2025-04-24 07:51:16,594 - duckdb-integrator - INFO - Logging query performance for source: postgres
2025-04-24 07:51:16,734 - duckdb-integrator - INFO - Query performance logged successfully
2025-04-24 07:51:16,992 - duckdb-integrator - INFO - Logging query performance for source: postgres
2025-04-24 07:51:17,119 - duckdb-integrator - INFO - Query performance logged successfully
2025-04-24 07:51:17,432 - duckdb-integrator - INFO - Logging query performance for source: duckdb
2025-04-24 07:51:17,485 - duckdb-integrator - INFO - Query performance logged successfully
2025-04-24 07:51:17,560 - duckdb-integrator - INFO - Starting duplicate column renaming process
2025-04-24 07:51:17,653 - duckdb-integrator - INFO - Found 3 duplicate columns: ['resource_id_1', 'client_id_1', 'project_id_1']
2025-04-24 07

,sk,client_id_bq_data,project_id_bq_data,resource_id_bq_data,project_name,CALENDAR__date,duration_hour,ts_start,ts_end,duration,id,resource_id_pg_data,client_id_pg_data,project_id_pg_data,pricing_client_id,date_from,date_to,pricing_delivery_id,pricing_value,total_amount
0,project_id:10<>resource_id:1<>week_start:2023-...,14,10,1,Meal Planner,2023-11-27,0,2023-11-27 01:00:00+01:00,2023-12-03 01:00:00+01:00,144.0,6,1,14,10,4,2023-01-01,NaT,<NA>,650.0,93600.0
1,project_id:22<>resource_id:3<>week_start:2023-...,7,22,3,[Svět Olejů] data-driven,2023-11-27,0,2023-11-27 01:00:00+01:00,2023-12-03 01:00:00+01:00,144.0,29,3,7,22,14,2023-01-01,NaT,2,0.0,0.0
2,project_id:23<>resource_id:3<>week_start:2023-...,7,23,3,[Neocity] data-driven,2023-11-27,0,2023-11-27 01:00:00+01:00,2023-12-03 01:00:00+01:00,144.0,30,3,7,23,14,2023-01-01,NaT,2,0.0,0.0
3,project_id:10<>resource_id:1<>week_start:2023-...,14,10,1,Meal Planner,2023-12-04,0,2023-12-04 01:00:00+01:00,2023-12-10 01:00:00+01:00,144.0,6,1,14,10,4,2023-01-01,NaT,<NA>,650.0,93600.0
4,project_id:22<>resource_id:3<>week_start:2023-...,7,22,3,[Svět Olejů] data-driven,2023-12-04,0,2023-12-04 01:00:00+01:00,2023-12-10 01:00:00+01:00,144.0,29,3,7,22,14,2023-01-01,NaT,2,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,project_id:23<>resource_id:3<>week_start:2025-...,7,23,3,[Neocity] data-driven,2025-04-14,0,2025-04-14 02:00:00+02:00,2025-04-20 02:00:00+02:00,144.0,17,3,7,23,8,2024-02-01,NaT,<NA>,2500.0,360000.0
619,project_id:8<>resource_id:3<>week_start:2025-0...,10,8,3,[ABCtech.cz] data-driven,2025-04-14,0,2025-04-14 02:00:00+02:00,2025-04-20 02:00:00+02:00,144.0,9,3,10,8,8,2023-12-01,NaT,<NA>,2500.0,360000.0
620,project_id:22<>resource_id:3<>week_start:2025-...,7,22,3,[Svět Olejů] data-driven,2025-04-21,0,2025-04-21 02:00:00+02:00,2025-04-27 02:00:00+02:00,144.0,16,3,7,22,8,2024-02-01,NaT,<NA>,2500.0,360000.0
621,project_id:23<>resource_id:3<>week_start:2025-...,7,23,3,[Neocity] data-driven,2025-04-21,0,2025-04-21 02:00:00+02:00,2025-04-27 02:00:00+02:00,144.0,17,3,7,23,8,2024-02-01,NaT,<NA>,2500.0,360000.0


In [10]:

logs = builder.get_performance_logs()
logs

2025-04-24 07:52:25,556 - join-advisor - INFO - Retrieving performance logs (start_time=None, end_time=None)
2025-04-24 07:52:25,675 - duckdb-integrator - INFO - Retrieving query logs (start_time=None, end_time=None)
2025-04-24 07:52:25,823 - duckdb-integrator - INFO - Retrieved 86 query log entries


,query_id,timestamp,source,query_text,execution_time_ms,rows_affected,execution_plan,error
0,604844284065454888,2025-04-24 07:51:17.432973,duckdb,"WITH\n bq_data AS (\n SELECT *, EXTR...",518.976,623,"{""explain_analyze"": [""analyzed_plan""]}",None
1,1234168737097055596,2025-04-24 07:51:16.993248,postgres,SELECT * FROM public.pricing_client,46.075,14,"{""result_set_size"": 0, ""cumulative_rows_scanne...",None
2,2119211808148212569,2025-04-24 07:51:16.594497,postgres,SELECT * FROM public.project_resources_pricings,46.716,25,"{""result_set_size"": 0, ""cumulative_rows_scanne...",None
3,8557502201837046327,2025-04-24 07:51:16.332319,bigquery,SELECT * FROM `datamancers.dbt.L1_POSTGRES__JO...,66.000,3331,"{""total_bytes_processed"": 0, ""total_bytes_bill...",None
4,3716656553504517145,2025-04-24 07:50:56.859756,duckdb,"WITH\n bq_data AS (\n SELECT *, EXTR...",322.073,0,{},"Parser Error: syntax error at or near ""AS"""
...,...,...,...,...,...,...,...,...
81,8015218326966216739,2025-04-15 17:33:14.378296,duckdb,SELECT * FROM POSTGRES(public.projects) LIMIT 5,998.272,0,{},"string indices must be integers, not 'str'"
82,5902853791757115161,2025-04-15 17:22:53.212313,duckdb,SELECT * FROM bq_temp_4565583168864758885 LIMIT 5,3556.648,5,"{""explain_analyze"": [""analyzed_plan""]}",None
83,8297740871016363555,2025-04-15 17:22:52.909441,bigquery,SELECT * FROM `datamancers.dbt.L2__JOBS__events`,96.000,3516,"{""total_bytes_processed"": 0, ""total_bytes_bill...",None
84,5233769583790885430,2025-04-15 17:22:49.363235,duckdb,SELECT * FROM postgres_db.public.projects LIMIT 5,1485.399,5,"{""explain_analyze"": [""analyzed_plan""]}",None
